In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder,LabelEncoder,StandardScaler
import pickle

In [3]:
### load the file

data=pd.read_csv('Churn_Modelling.csv')
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [5]:
### remove the unesccary data


data=data.drop(['RowNumber','CustomerId','Surname'],axis=1)
data.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [6]:
### convert the cetagorical value into numerical value

label_encoder_gender=LabelEncoder()
data['Gender']=label_encoder_gender.fit_transform(data['Gender'])
data.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,0,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,0,41,1,83807.86,1,0,1,112542.58,0
2,502,France,0,42,8,159660.80,3,1,0,113931.57,1
3,699,France,0,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,0,43,2,125510.82,1,1,1,79084.10,0


In [13]:
### convert the geographical value to numerical value

one_hot_geo=OneHotEncoder(handle_unknown='ignore')
geo_encoded=one_hot_geo.fit_transform(data[['Geography']]).toarray()
geo_encoded_df=pd.DataFrame(geo_encoded,columns=one_hot_geo.get_feature_names_out(['Geography']))
geo_encoded_df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0
...,...,...,...
9995,1.0,0.0,0.0
9996,1.0,0.0,0.0
9997,1.0,0.0,0.0
9998,0.0,1.0,0.0


In [14]:
### concat the one hot encoded geography data


data=pd.concat([data.drop('Geography',axis=1),geo_encoded_df],axis=1)
data.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [15]:
#### split the data into features and target

x=data.drop('EstimatedSalary',axis=1) ### independent features
y=data['EstimatedSalary'] ## dependent features

In [17]:
### create the training and testing set data

x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.1,random_state=42)


### standardscaler


scaler=StandardScaler()
x_train=scaler.fit_transform(x_train)
x_test=scaler.fit_transform(x_test)

In [18]:
#### save the enocders and pickle files


with open('label_encoder_gender.pkl','wb') as file:
    pickle.dump(label_encoder_gender,file)

with open('one_hot_geo.pkl','wb') as file:
    pickle.dump(one_hot_geo,file)

with open('scaler.pkl','wb') as file:
    pickle.dump(scaler,file)

In [19]:
### ANN regression

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [25]:
### build the model 
model=Sequential([
    Dense(64,activation='relu',input_shape=(x_train.shape[1],)), ##hl1
    Dense(32,activation='relu'), ## hl2 
    Dense(1)  ##op

])


### compile the model

model.compile(optimizer='adam',loss='mean_absolute_error',metrics=['mae'])
model.summary()

c:\Users\user\Desktop\ANN_CLASSIFICATION\env\Lib\site-packages\keras\src\layers\core\dense.py:106: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_15 (Dense)                │ (None, 64)             │           832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_16 (Dense)                │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_17 (Dense)                │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,945 (11.50 KB)

 Trainable params: 2,945 (11.50 KB)

 Non-trainable params: 0 (0.00 B)

In [26]:
### saving the logs 

from tensorflow.keras.callbacks import EarlyStopping,TensorBoard
import datetime


logs="regressionlogs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback=TensorBoard(log_dir=logs,histogram_freq=1)

In [31]:
### earlystopping

earlystopping_callback=EarlyStopping(monitor="val_loss",patience=5,restore_best_weights=True)


In [32]:
### train the model

history=model.fit(x_train,y_train,validation_data=(x_test,y_test),epochs=100,
          callbacks=[tensorboard_callback,earlystopping_callback]
)

Epoch 1/100
282/282 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 98616.3594 - mae: 98616.3594 - val_loss: 99498.0781 - val_mae: 99498.0781
Epoch 2/100
282/282 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 94699.1641 - mae: 94699.1641 - val_loss: 93779.3203 - val_mae: 93779.3203
Epoch 3/100
282/282 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 87383.4141 - mae: 87383.4141 - val_loss: 84761.2109 - val_mae: 84761.2109
Epoch 4/100
282/282 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 77574.8359 - mae: 77574.8359 - val_loss: 74062.5703 - val_mae: 74062.5703
Epoch 5/100
282/282 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 67372.5156 - mae: 67372.5156 - val_loss: 64236.6445 - val_mae: 64236.6445
Epoch 6/100
282/282 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 58998.6172 - mae: 58998.6172 - val_loss: 56928.8438 - val_mae: 56928.8438
Epoch 7/100
282/282 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 53694.4414 - mae: 53694.4414 - val_loss: 52988.5977 - val_mae: 52988.5977
Epoch 8/100
282/282 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - 

In [34]:
### load tensorboard


%load_ext tensorboard

In [36]:
### launch the tensorboard

%tensorboard --logdir regressionlogs/fit

Reusing TensorBoard on port 6006 (pid 704), started 0:00:10 ago. (Use '!kill 704' to kill it.)

In [37]:
### evaluate model on the test data

test_loss,test_mae=model.evaluate(x_test,y_test)

print(f'my test MAE is: {test_mae}')
print(f'the test loss is: {test_loss}')

32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 50094.9492 - mae: 50094.9492
my test MAE is: 50094.94921875
the test loss is: 50094.94921875


In [38]:
### save the regression file

model.save("regression.h5")